In [1]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
import numpy as np

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

from monai.networks.nets import  DenseNet121
from pe.datatasets.data_loaders.rsna_pe_simple import NiftiDataset
from torch.utils.data import  DataLoader
import torch 
import torch
from torch.utils.data import DataLoader, Subset, random_split

In [2]:
pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

# Define transforms
train_transforms = Compose([ScaleIntensity(), Resize((224, 224, 200)), RandRotate90()])

# Define Nifti dataset and data loader for the new dataset
csv_file = '/share/pi/nigam/data/RSNAPE/simplified_labels/train.csv'
root_dir = '/share/pi/nigam/data/RSNAPE/nifti_crop/'

#nifti_dataset = NiftiDataset(csv_file=csv_file, root_dir=root_dir,transform=train_transforms)
nifti_dataset = NiftiDataset(csv_file=csv_file, root_dir=root_dir)
loader        = DataLoader(nifti_dataset, batch_size=1, shuffle=True)

dataset_size = len(loader.dataset)
test_size    = int(0.2 * dataset_size)
train_size   = dataset_size - test_size

# Create indices for train and test subsets
indices = list(range(dataset_size))
train_indices, test_indices = indices[:train_size], indices[train_size:]

# Create Subset data loaders for train and test
train_loader = DataLoader(Subset(loader.dataset, train_indices), batch_size=1, shuffle=True)
test_loader  = DataLoader(Subset(loader.dataset, test_indices), batch_size=1, shuffle=False)

# Create a model (Swin or DenseNet)
model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=1).to(device)

# Create loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer     = torch.optim.Adam(model.parameters(), 1e-4)

MONAI version: 1.3.0
Numpy version: 1.25.2
Pytorch version: 2.0.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /share/pi/nigam/alejandro/envs/pe_env/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: 0.21.0
scipy version: 1.9.3
Pillow version: 10.0.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2+cu117
tqdm version: 4.66.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.1.0
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: 

In [3]:
for batch_data in train_loader:
    inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
    break

In [4]:
inputs_a = torch.unsqueeze(train_transforms(inputs),dim=0)
outputs = model(inputs_a)

In [10]:
import torch.nn.functional as F 

In [11]:
F.binary_cross_entropy_with_logits(outputs,labels)

ValueError: Target size (torch.Size([1])) must be the same as input size (torch.Size([1, 1]))

In [7]:
loss_function( ,)

metatensor([[0.3415]], grad_fn=<AliasBackward0>)

In [26]:
# start a typical PyTorch training
val_interval      = 2
best_metric       = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values     = []
max_epochs        = 5

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        inputs         = torch.unsqueeze(train_transforms(inputs),dim=0)
        optimizer.zero_grad()
        # import pdb;pdb.set_trace()
        outputs  = model(inputs)
        loss     = loss_function(outputs,  labels.reshape_as(outputs).float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_len   = train_size // train_loader.batch_size
        print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
    

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()

        num_correct = 0.0
        metric_count = 0
        for val_data in test_loader:
            val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
            with torch.no_grad():
                val_outputs = model(val_images)
                value = torch.eq(val_outputs.argmax(dim=1), val_labels.argmax(dim=1))
                metric_count += len(value)
                num_correct += value.sum().item()

        metric = num_correct / metric_count
        metric_values.append(metric)

        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_array.pth")
            print("saved new best metric model")

        print(f"Current epoch: {epoch+1} current accuracy: {metric:.4f} ")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

----------
epoch 1/5
1/387, train_loss: -0.0000
2/387, train_loss: -0.0000
3/387, train_loss: -0.0000
4/387, train_loss: -0.0000
5/387, train_loss: -0.0000
6/387, train_loss: -0.0000
7/387, train_loss: -0.0000
8/387, train_loss: -0.0000
9/387, train_loss: -0.0000
10/387, train_loss: -0.0000
11/387, train_loss: -0.0000
12/387, train_loss: -0.0000
13/387, train_loss: -0.0000
14/387, train_loss: -0.0000
15/387, train_loss: -0.0000
16/387, train_loss: -0.0000
17/387, train_loss: -0.0000
18/387, train_loss: -0.0000
19/387, train_loss: -0.0000
20/387, train_loss: -0.0000
21/387, train_loss: -0.0000
